In [219]:
import csv
import pandas as pd
import re
import os
import glob

#### Life expectancy

In [187]:
life_exp_2019 = "data\health\life_2012_2019_data.csv"
life_exp_2020 = "data\health\life_2020_data.csv"

In [188]:
life_exp_2019 = pd.read_csv(life_exp_2019)
life_exp_2019

,measure_id,measure_name,location_id,location_name,state_name,race_ethnicity_group,sex_id,sex_name,age_group_id,age_group_name,year_id,metric_id,metric_name,val,upper,lower
0,26,Life expectancy,523,Alabama,Alabama,State-level,1,Male,73,25,1990,5,Years,47.401899,47.663865,47.131186
1,26,Life expectancy,523,Alabama,Alabama,State-level,1,Male,73,25,1991,5,Years,47.293256,47.535055,47.033662
2,26,Life expectancy,523,Alabama,Alabama,State-level,1,Male,73,25,1992,5,Years,47.667564,47.935554,47.409149
3,26,Life expectancy,523,Alabama,Alabama,State-level,1,Male,73,25,1993,5,Years,47.826836,48.098470,47.571233
4,26,Life expectancy,523,Alabama,Alabama,State-level,1,Male,73,25,1994,5,Years,47.718326,47.982345,47.456253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627295,14,All-cause mortality rate,53912,"Wyoming; Non-Hispanic, White",Wyoming,"Non-Hispanic, White",2,Female,160,85 plus,2015,3,Rate,0.130016,0.134438,0.125880
627296,14,All-cause mortality rate,53912,"Wyoming; Non-Hispanic, White",Wyoming,"Non-Hispanic, White",2,Female,160,85 plus,2016,3,Rate,0.122373,0.126349,0.118465
627297,14,All-cause mortality rate,53912,"Wyoming; Non-Hispanic, White",Wyoming,"Non-Hispanic, White",2,Female,160,85 plus,2017,3,Rate,0.124795,0.129249,0.120578
627298,14,All-cause mortality rate,53912,"Wyoming; Non-Hispanic, White",Wyoming,"Non-Hispanic, White",2,Female,160,85 plus,2018,3,Rate,0.132197,0.136809,0.127338


In [189]:
life_2019 = life_exp_2019.loc[life_exp_2019['measure_name'] == 'Life expectancy']\
    [life_exp_2019['age_group_name'] == 'Birth'][life_exp_2019['year_id'] >= 2011]\
    [life_exp_2019['race_ethnicity_group'] >= 'State-level']
life_2019

C:\Users\18973\AppData\Local\Temp\ipykernel_25176\65814213.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  life_2019 = life_exp_2019.loc[life_exp_2019['measure_name'] == 'Life expectancy']\


,measure_id,measure_name,location_id,location_name,state_name,race_ethnicity_group,sex_id,sex_name,age_group_id,age_group_name,year_id,metric_id,metric_name,val,upper,lower
141,26,Life expectancy,523,Alabama,Alabama,State-level,1,Male,164,Birth,2011,5,Years,72.763545,73.028044,72.477746
142,26,Life expectancy,523,Alabama,Alabama,State-level,1,Male,164,Birth,2012,5,Years,72.792684,73.066355,72.506043
143,26,Life expectancy,523,Alabama,Alabama,State-level,1,Male,164,Birth,2013,5,Years,72.627114,72.893336,72.372241
144,26,Life expectancy,523,Alabama,Alabama,State-level,1,Male,164,Birth,2014,5,Years,73.008197,73.280054,72.753815
145,26,Life expectancy,523,Alabama,Alabama,State-level,1,Male,164,Birth,2015,5,Years,72.705515,72.998800,72.438260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9175,26,Life expectancy,573,Wyoming,Wyoming,State-level,2,Female,164,Birth,2015,5,Years,81.172748,81.662615,80.658190
9176,26,Life expectancy,573,Wyoming,Wyoming,State-level,2,Female,164,Birth,2016,5,Years,81.631941,82.165660,81.089740
9177,26,Life expectancy,573,Wyoming,Wyoming,State-level,2,Female,164,Birth,2017,5,Years,81.358672,81.879027,80.809047
9178,26,Life expectancy,573,Wyoming,Wyoming,State-level,2,Female,164,Birth,2018,5,Years,81.546103,82.103023,80.992780


In [190]:
def gender_to_ratio(df):
    new_df = df[df['sex_name'] == "Female"][["state_name", "sex_name", "year_id", "val"]].reset_index()
    m_df = df[df['sex_name'] == "Male"][["state_name", "sex_name", "year_id", "val"]].reset_index()
    new_df['life_exp_m_f'] = m_df['val'].divide(new_df['val'])
    new_df = new_df.drop(columns=['sex_name', 'val']).reset_index(drop=True)
    new_df = new_df.rename({'state_name':'state', 'year_id':'year'}, axis=1)
    return new_df

In [191]:
life_df_2019 = gender_to_ratio(life_2019)
life_df_2019 = life_df_2019.drop(columns='index')
life_df_2019.drop(life_df_2019[(life_df_2019['state'] == 'District of Columbia')].index, inplace=True)
life_df_2019

,state,year,life_exp_m_f
0,Alabama,2011,0.928918
1,Alabama,2012,0.931252
2,Alabama,2013,0.927294
3,Alabama,2014,0.933043
4,Alabama,2015,0.930030
...,...,...,...
454,Wyoming,2015,0.947130
455,Wyoming,2016,0.945792
456,Wyoming,2017,0.955632
457,Wyoming,2018,0.946401


Combine 2020 data

In [213]:
''' 
data from a pdf: 
        convert to Excel first, do some data wrangling 
        export as a csv
'''
life_2020 = pd.read_csv(life_exp_2020)
life_2020 = life_2020.drop(columns=life_2020.columns[-1]).reset_index(drop=True)
life_2020['year_id'] = life_2020['year_id'].astype(int)
life_2020['state_name'] = life_2020['state_name'].apply(lambda x: x.strip())
life_2020.head()

,state_name,val,sex_name,year_id
0,Alabama,70.1,Male,2020
1,Alabama,76.4,Female,2020
2,Alaska,74.3,Male,2020
3,Alaska,79.2,Female,2020
4,Arizona,73.4,Male,2020


In [214]:
life_df_2020 = gender_to_ratio(life_2020)
life_df_2020.drop(columns='index', inplace=True)
life_df_2020.drop(8, inplace=True)   # remove state == "District of Columbia"
life_df_2020.drop(44, inplace=True)   # remove state == "United States"
life_df_2020 = life_df_2020.reset_index(drop=True)
life_df_2020

,state,year,life_exp_m_f
0,Alabama,2020,0.917539
1,Alaska,2020,0.938131
2,Arizona,2020,0.923270
3,Arkansas,2020,0.928198
4,California,2020,0.929268
5,Colorado,2020,0.936959
6,Connecticut,2020,0.929889
7,Delaware,2020,0.929560
8,Florida,2020,0.926708
9,Georgia,2020,0.929757


In [215]:
life_exp = pd.concat([life_df_2019, life_df_2020]).reset_index(drop=True)
life_exp

,state,year,life_exp_m_f
0,Alabama,2011,0.928918
1,Alabama,2012,0.931252
2,Alabama,2013,0.927294
3,Alabama,2014,0.933043
4,Alabama,2015,0.930030
...,...,...,...
495,Virginia,2020,0.937578
496,Washington,2020,0.942402
497,West Virginia,2020,0.917214
498,Wisconsin,2020,0.936488


In [227]:
life_exp = life_exp.sort_values(['state', 'year'], ascending=[True, True]).reset_index(drop=True)
life_exp

,state,sex_ratio,year
0,Alabama,93.8,2011
1,Alabama,94.0,2012
2,Alabama,94.0,2013
3,Alabama,93.9,2014
4,Alabama,93.9,2015
...,...,...,...
495,Wyoming,103.8,2016
496,Wyoming,104.3,2017
497,Wyoming,102.6,2018
498,Wyoming,104.7,2019


#### Sex ratio (males per 100 females)

In [195]:
import re

In [196]:
def remove_after_exclamation(s):
    return re.sub('!.+', '', s)

In [197]:
def get_sr_df(df, year):
    ''' Transpose a row to a dataframe '''
    sr_df = df[df['Label (Grouping)'].str.contains(r'Sex ratio')].T.reset_index() # get the specific row
    sr_df.columns = sr_df.iloc[0]
    sr_df = sr_df.drop(sr_df.index[0])   # reset column names
    sr_df.columns = {'state': 'Label (Grouping)', 'sex_ratio': 'Sex ratio (males per 100 females)'}
    sr_df['state'] = sr_df['state'].apply(remove_after_exclamation)
    sr_df.drop(sr_df[(sr_df['sex_ratio'] == '(X)')].index, inplace=True)
    sr_df.drop(sr_df[(sr_df['state'] == 'District of Columbia')].index, inplace=True)
    sr_df.drop(sr_df[(sr_df['state'] == 'Puerto Rico')].index, inplace=True)
    sr_df['year'] = year

    return sr_df.reset_index(drop=True)

In [228]:
folder_path = 'data/health/sex_ratio/' # Replace with the path to your folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

sex_ratio = pd.DataFrame()
year = 2011
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    temp_df = get_sr_df(pd.read_csv(file_path), year)
    year += 1
    sex_ratio = pd.concat([sex_ratio, temp_df]).reset_index(drop=True)
sex_ratio

,state,sex_ratio,year
0,Alabama,93.8,2011
1,Alaska,106.6,2011
2,Arizona,99.2,2011
3,Arkansas,97.2,2011
4,California,99.0,2011
...,...,...,...
495,Virginia,96.8,2020
496,Washington,100.1,2020
497,West Virginia,97.8,2020
498,Wisconsin,99.1,2020


In [229]:
health_data = life_exp.merge(sex_ratio, left_on=['state','year'], right_on=['state','year'])
health_data

,state,sex_ratio_x,year,sex_ratio_y
0,Alabama,93.8,2011,93.8
1,Alabama,94.0,2012,94.0
2,Alabama,94.0,2013,94.0
3,Alabama,93.9,2014,93.9
4,Alabama,93.9,2015,93.9
...,...,...,...,...
495,Wyoming,103.8,2016,103.8
496,Wyoming,104.3,2017,104.3
497,Wyoming,102.6,2018,102.6
498,Wyoming,104.7,2019,104.7


In [218]:
health_data.to_csv("health_data.csv", index=False)